In [69]:
# 라이브러리 불러오기
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torchvision import transforms
import torchaudio
import pandas as pd

In [70]:
from GPU_torch import GPU
device = GPU()
print("current device : ",device)
print(device)

cant use gpu , activating cpu
current device :  cpu
cpu


In [71]:
# Dataset 클래스 정의
class UrbanSoundDataset(Dataset):
    def __init__(self, annotations_file, audio_dir, transformation, target_sample_rate):
        self.annotations = (annotations_file)
        self.audio_dir = audio_dir
        self.transformation = transformation
        self.target_sample_rate = target_sample_rate

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, label

    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.target_sample_rate:
            signal = signal[:, :self.target_sample_rate]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.target_sample_rate:
            num_missing_samples = self.target_sample_rate - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[index, 0])
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]


In [72]:
# 데이터셋 및 데이터 로더 설정



# Load the dataset
try:
    ANNOTATIONS_FILE = pd.read_csv('/Users/cafalena/sound_datasets/urbansound8k/UrbanSound8K/metadata/UrbanSound8K.csv')
except:
    try:
        ANNOTATIONS_FILE = pd.read_csv('/Users/owo/sound_datasets/urbansound8k/UrbanSound8K/metadata/UrbanSound8K.csv')
    except:
        ANNOTATIONS_FILE = pd.read_csv('C:/Users/PC/AppData/@FOLDER/@Project/UrbanSound8K/metadata/UrbanSound8K.csv')



AUDIO_DIR = "/Users/owo/sound_datasets/urbansound8k/UrbanSound8K/audio"##cafalena or owo
SAMPLE_RATE = 22050
BATCH_SIZE = 128
NUM_WORKERS = 4
PIN_MEMORY = True if torch.cuda.is_available() else False

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

transformation = transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=128),
    torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80)
])



usd = UrbanSoundDataset(ANNOTATIONS_FILE, AUDIO_DIR, transformation, SAMPLE_RATE)

# 데이터셋 분리
dataset_size = len(usd)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = torch.utils.data.random_split(usd, [train_size, test_size])

# 데이터 로더 생성
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=NUM_WORKERS,
                          pin_memory=PIN_MEMORY)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=NUM_WORKERS,
                         pin_memory=PIN_MEMORY)



#down is when no test data
# data_loader = torch.utils.data.DataLoader(
#     usd,
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     drop_last=False,
#     num_workers=NUM_WORKERS,
#     pin_memory=PIN_MEMORY
# )


/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [73]:
# ResNet18 모델 설정
model = resnet18(pretrained=False)
model.fc = nn.Linear(512, 10)  # UrbanSound8K의 클래스 개수인 10으로 변경
model = model.to(device)


/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [74]:
# 손실함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [75]:
# 훈련 함수 정의
def train(model, data_loader, criterion, optimizer, device,NB_EPOCH):

    for epoch in range(NB_EPOCHS):
        for i, (inputs, labels) in enumerate(data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print (f'Epoch [{epoch+1}/{epoch}], Step [{i+1}/{len(data_loader)}], Loss: {loss.item()}')


In [76]:
# 훈련 시작

train(model, train_loader, criterion, optimizer, device,NB_EPOCH=10)


KeyboardInterrupt: 

In [ ]:
# 모델 평가 함수 정의
def evaluate_model(model, test_loader, device):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on test images: {100 * correct / total}%')


In [ ]:
# 훈련 및 평가
EPOCHS = 10
train(model, train_loader, criterion, optimizer, device)
evaluate_model(model, test_loader, device)
